In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed_PPA')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed_PPA')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed_PPA')

In [3]:
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2

In [4]:
import torch
def compute_loss(sub_df,test_data):
    pred = torch.Tensor(np.log(sub_df.loc[:,['A','B','NEITHER']].values))
    label = torch.LongTensor(list(test_data.label))
    loss = torch.nn.NLLLoss()
    loss_value = loss(pred,label).item()
    return loss_value

In [5]:
train_data = gap_train.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
train_data.A_vector = train_data.A_vector.map(lambda x:np.mean(x,axis = 0))
train_data.B_vector = train_data.B_vector.map(lambda x:np.mean(x,axis = 0))
train_data.pron_vector = train_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
train_data["product_vector_A"] = train_data.A_vector*train_data.pron_vector
train_data["product_vector_B"] = train_data.B_vector*train_data.pron_vector
train_data["label"] = train_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
train_data = train_data.drop(columns= ["A-coref","B-coref"])

In [6]:
test_data = gap_test.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
test_data.A_vector = test_data.A_vector.map(lambda x:np.mean(x,axis = 0))
test_data.B_vector = test_data.B_vector.map(lambda x:np.mean(x,axis = 0))
test_data.pron_vector = test_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
test_data["product_vector_A"] = test_data.A_vector*test_data.pron_vector
test_data["product_vector_B"] = test_data.B_vector*test_data.pron_vector
test_data["label"] = test_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
test_data = test_data.drop(columns= ["A-coref","B-coref"])

In [7]:
valid_data = gap_valid.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
valid_data.A_vector = valid_data.A_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.B_vector = valid_data.B_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.pron_vector = valid_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
valid_data["product_vector_A"] = valid_data.A_vector*valid_data.pron_vector
valid_data["product_vector_B"] = valid_data.B_vector*valid_data.pron_vector
valid_data["label"] = valid_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
valid_data = valid_data.drop(columns= ["A-coref","B-coref"])

In [8]:
columns = train_data.columns[:-1]
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = list(train_data.label)   

In [9]:
X_valid = np.concatenate([np.array(list(valid_data[col])).reshape(valid_data.shape[0],-1) for col in columns],axis = 1)
y_valid = list(valid_data.label)   

In [10]:
X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test= list(test_data.label)   

In [11]:
train_data.head()

,A_dist,B_dist,A_pos,B_pos,pron_pos,A_vector,B_vector,pron_vector,product_vector_A,product_vector_B,label
0,0.014,0.008,0.444444,0.464052,0.490196,"[-0.010048583, -0.22500569, -0.18638803, 0.246...","[-0.064268865, -0.14198042, -0.077147625, -0.0...","[-0.35196483, -0.22052056, 0.4001546, -0.44971...","[0.0035367478, 0.04961838, -0.07458403, -0.110...","[0.02262038, 0.0313096, -0.030870976, 0.025482...",1
1,0.026,0.014,0.253623,0.275362,0.300725,"[-0.033601925, -0.428665, 0.29058167, -0.17485...","[0.40463662, 0.12124419, -0.07090483, -0.07563...","[-0.051942207, 0.44361016, 0.1704489, 0.145203...","[0.0017453581, -0.19016016, 0.049529325, -0.02...","[-0.021017719, 0.053785156, -0.01208565, -0.01...",0
2,0.024,0.008,0.408333,0.475000,0.508333,"[0.51285046, -0.3347876, -0.28261185, -0.00890...","[-0.049855076, 0.0751394, 0.29557943, -0.32997...","[0.08574369, 0.23280331, 0.4195495, 0.34910136...","[0.04397369, -0.07793967, -0.11856966, -0.0031...","[-0.004274758, 0.017492702, 0.124010205, -0.11...",0
3,0.056,-0.004,0.222951,0.321311,0.314754,"[0.05159696, -0.16686673, -0.35011744, -0.7497...","[0.18233861, -0.36514157, 0.14521956, -0.57738...","[-0.13253343, -0.44861144, -0.1258747, -0.0025...","[-0.0068383217, 0.07485832, 0.04407093, 0.0019...","[-0.024165962, 0.16380669, -0.018279469, 0.001...",1
4,-0.006,-0.056,0.473118,0.607527,0.456989,"[-0.1981886, 0.28381836, 0.4610955, 0.2881973,...","[0.022472247, -0.05395194, -0.18112913, 0.1082...","[-0.37515718, 0.1776664, -0.034614205, -0.3140...","[0.07435188, 0.050424986, -0.015960455, -0.090...","[-0.008430625, -0.009585447, 0.0062696408, -0....",0


In [13]:
import pickle
f = open( "./temp_result/base_model_data_PPA", "wb" )
pickle.dump(X_train,  f)
pickle.dump(y_train,  f)
pickle.dump(X_valid,  f)
pickle.dump(y_valid,  f)
pickle.dump(X_test,  f)
f.close()

In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)


pred_lr = lr.predict_proba(X_test)

sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr

sub_df.head()


sub_df.to_csv("./test_and_submit/submission+model+lr@"+str(datetime.datetime.now())+".csv", index=False)

In [15]:
compute_loss(sub_df,test_data)

0.8081879019737244

In [16]:
from sklearn.svm import SVC
svm = SVC(C = 7.0,verbose=True,probability = True,gamma = "auto").fit(X_train, y_train)


pred_svm = svm.predict_proba(X_test)

sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_svm


[LibSVM]

In [17]:
sub_df.to_csv("./test_and_submit/submission+model+svm@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.689653,0.172671,0.137675
1,development-2,0.877513,0.027717,0.094770
2,development-3,0.039383,0.923054,0.037563
3,development-4,0.328847,0.589709,0.081444
4,development-5,0.076487,0.852824,0.070689


In [18]:
compute_loss(sub_df,test_data)

0.5735058188438416